<a href="https://colab.research.google.com/github/PANDASANG1231/Deeplearning_byHand/blob/master/029_ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
sys.path.append('/content/drive/MyDrive/Colab Notebooks/deeplearning_note')
from tool import *

Mounted at /content/drive


In [2]:
! nvidia-smi

Mon Jan 17 09:38:08 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P0    55W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import torch
import torchvision

In [4]:
class Resnet_block(torch.nn.Module):

  def __init__(self, in_channels, out_channels, if_double, if_conv):

    super().__init__()
    self.if_double = if_double
    self.if_conv = if_conv
    self.conv2d_3 = None

    if if_double:
      self.conv2d_1 = torch.nn.Conv2d(in_channels=in_channels, out_channels=out_channels, stride=2, kernel_size=3, padding=1)
      self.conv2d_2 = torch.nn.Conv2d(in_channels=out_channels, out_channels=out_channels, stride=1, kernel_size=3, padding=1)

      if if_conv:
        self.conv2d_3 = torch.nn.Conv2d(in_channels=in_channels, out_channels=out_channels, stride=2, kernel_size=1, padding=0)

    else:
      self.conv2d_1 = torch.nn.Conv2d(in_channels=in_channels, out_channels=out_channels, stride=1, kernel_size=3, padding=1)
      self.conv2d_2 = torch.nn.Conv2d(in_channels=out_channels, out_channels=out_channels, stride=1, kernel_size=3, padding=1)

      if if_conv:
        self.conv2d_3 = torch.nn.Conv2d(in_channels=in_channels, out_channels=out_channels, stride=1, kernel_size=1, padding=0)

    self.bn1 = torch.nn.BatchNorm2d(num_features=out_channels)
    self.bn2 = torch.nn.BatchNorm2d(num_features=out_channels)
    self.relu = torch.nn.ReLU()

  def forward(self, X):

    Y = self.relu(self.bn1(self.conv2d_1(X)))
    Y = self.bn2(self.conv2d_2(Y))

    if self.conv2d_3:
      X = self.conv2d_3(X)
    
    Y += X
    return self.relu(Y)


Training with Resnet

In [5]:
train_dataset = torchvision.datasets.FashionMNIST(root="./",train=True,download=True,transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor(), torchvision.transforms.Resize(96)]))
test_dataset = torchvision.datasets.FashionMNIST(root="./",train=False,download=True,transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor(), torchvision.transforms.Resize(96)]))

# train_dataset = torchvision.datasets.FashionMNIST(root="./",train=True,download=True,transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor()]))
# test_dataset = torchvision.datasets.FashionMNIST(root="./",train=False,download=True,transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor()]))

In [10]:
batch_size = 256

ResNet = torch.nn.Sequential(
    *[
     torch.nn.Sequential(torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3), torch.nn.BatchNorm2d(64), 
                         torch.nn.ReLU(), torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=1)), 
     torch.nn.Sequential(*[Resnet_block(64, 64, False, True), Resnet_block(64, 64, False, True)]), 
     torch.nn.Sequential(*[Resnet_block(64, 128, True, True), Resnet_block(128, 128, False, True)]), 
     torch.nn.Sequential(*[Resnet_block(128, 256, True, True), Resnet_block(256, 256, False, True)]), 
     torch.nn.Sequential(*[Resnet_block(256, 512, True, True), Resnet_block(512, 512, False, True)]), 
     torch.nn.AdaptiveAvgPool2d((1,1)),
     torch.nn.Flatten(),
     torch.nn.Linear(512, 10),
    ]
)


train_dataloader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=batch_size,
                                               shuffle=True,
                                               num_workers=4)

test_dataloader = torch.utils.data.DataLoader(test_dataset,
                                              batch_size=batch_size,
                                              shuffle=False,
                                              num_workers=4)


device = torch.device('cuda')

train_p2(num_epochs=10, 
         net=ResNet,  
         lr=0.05, 
         train_iter=train_dataloader,
         test_iter=test_dataloader,
         device=device,
         optim_type="SGD")


training on cuda
loss 0.481, train acc 0.824, test acc 0.826
loss 0.266, train acc 0.902, test acc 0.840
loss 0.207, train acc 0.923, test acc 0.880
loss 0.158, train acc 0.943, test acc 0.876
loss 0.122, train acc 0.956, test acc 0.905
loss 0.093, train acc 0.968, test acc 0.892
loss 0.070, train acc 0.975, test acc 0.884
loss 0.051, train acc 0.982, test acc 0.904
loss 0.038, train acc 0.987, test acc 0.903
loss 0.025, train acc 0.992, test acc 0.889
2149.6 examples/sec on cuda


In [7]:
X = torch.ones((1, 1, 96, 96))

ResNet = torch.nn.Sequential(
    *[
     torch.nn.Sequential(*[Resnet_block(1, 64, True, True), Resnet_block(64, 64, False, True), 
                           Resnet_block(64, 64, False, True)]), 
     torch.nn.Sequential(*[Resnet_block(64, 128, True, True), Resnet_block(128, 128, False, True), 
                           Resnet_block(128, 128, False, True)]), 
     torch.nn.Sequential(*[Resnet_block(128, 256, True, True), Resnet_block(256, 256, False, True), 
                           Resnet_block(256, 256, False, True)]), 
     torch.nn.Sequential(*[Resnet_block(256, 512, True, True), Resnet_block(512, 512, False, True), 
                           Resnet_block(512, 512, False, True)]), 
     torch.nn.AdaptiveAvgPool2d((1, 1)),
     torch.nn.Flatten(),
     torch.nn.Linear(512, 10),
    ]
)

for layer in ResNet:
    X = layer(X)
    print(layer.__class__.__name__, X.shape)

Sequential torch.Size([1, 64, 48, 48])
Sequential torch.Size([1, 128, 24, 24])
Sequential torch.Size([1, 256, 12, 12])
Sequential torch.Size([1, 512, 6, 6])
AdaptiveAvgPool2d torch.Size([1, 512, 1, 1])
Flatten torch.Size([1, 512])
Linear torch.Size([1, 10])
